<a href="https://colab.research.google.com/github/SojeongAn/tokyo/blob/main/Precipitation_type_Classification_(include_Polrolniczak).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI for precipitation detection (Baseline by Polrolniczak 2021).*

This colab contains:
* Code to read the dataset using [sklearn and Deep learning module (Pytorch, Tensorflow)]
* Example code to load this model and use it to make predictions ([github](https://github.com/SojeongAn/tokyo)).

It has been tested in a public Google colab kernel.

## Access your google drive


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## Library dependency installs and imports



In [ ]:
# BASIC LIBRARIES 
import os
import pickle
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import netCDF4 as nc

In [ ]:
# SKLEARN MODEL
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

## Dataset Load and pre-processing

In [ ]:
class snowData():

    def __init__(
        self
    ):
        self.diff = dt.timedelta(hours=9)
        self.interval = dt.timedelta(hours=1)


    def readData(self, fname):
        with open(fname):
            file = nc.Dataset(fname,mode='r')
            for v in file.variables.keys():
                var = file.variables[v]
                print(var)


    def loadingData(self, year, month):
        if month == 12:
            day_count = 31
        else:
            day_count = (dt.date(year, month+1, 1) - dt.date(year, month, 1)).days
        
        for i in range(0, day_count):
            day = i+1

            for j in range(0, 6):
                start_date = dt.datetime(year, month, day, 0, 0)
                kst = (start_date + j * self.interval) 
                utc = kst - self.diff

                ym = utc.strftime("%Y%m")
                ymdhm = utc.strftime("%Y%m%d_%H%M")
                mlfile = 'data/{}/ERA5_anal_ml_{}.nc'.format(ym, ymdhm)
                sfcfile = 'data/{}/ERA5_anal_sfc_{}.nc'.format(ym, ymdhm)

                if not os.path.isfile(sfc):
                    pass

                ml = self.mlDataset(mlfile)
                sfc = self.sfcDataset(sfcfile)

                data = ml.update(sfc)
                ymdh = kst.strftime("%Y%m%d%H")
                save_path = os.path.join('data/prepro', ymdh + '.pickle')
                with open(save_path,'wb') as fw:
                    pickle.dump(data, fw)


    def mlDataset(self, fname):

        with open(fname):
            ds = nc.Dataset(fname, mode='r')
            time = ds.variables['time']
            time_ = nc.num2date(time[:], time.units, time.calendar)
            time_ = dt.datetime.strptime(str(time_[0]),'%Y-%m-%d %H:%M:%S')
            time_ = (time_+self.diff).strftime("%Y%m%d%H%M")
            crwc = ds.variables['crwc'][:].reshape(-1)
            cswc = ds.variables['cswc'][:].reshape(-1)
            etadot = ds.variables['etadot'][:].reshape(-1)
            z = ds.variables['z'][:].reshape(-1)
            t = ds.variables['t'][:].reshape(-1)
            q = ds.variables['q'][:].reshape(-1)
            w = ds.variables['w'][:].reshape(-1)
            vo = ds.variables['vo'][:].reshape(-1)
            lnsp = ds.variables['lnsp'][:].reshape(-1)
            d = ds.variables['d'][:].reshape(-1)
            u = ds.variables['u'][:].reshape(-1)
            v = ds.variables['v'][:].reshape(-1)
            o3 = ds.variables['o3'][:].reshape(-1)
            clwc = ds.variables['clwc'][:].reshape(-1)
            ciwc = ds.variables['ciwc'][:].reshape(-1)
            cc = ds.variables['cc'][:].reshape(-1)
        mldict = {
                'time': time_, 'crwc': crwc, 'cswc': cswc, 
                'etadot': etadot, 'z': z, 't': t, 'q': q, 'w': w, 
                'vo': vo, 'lnsp': lnsp, 'd': d, 'u': u, 'v': v, 
                'o3': o3, 'clwc': clwc, 'ciwc': ciwc, 'cc': cc
                }
        return mldict


    def sfcDataset(self, fname):
        with open(fname):
            ds = nc.Dataset(fname, mode='r')
            lsm = ds.variables['lsm'][:]
            siconc = ds.variables['siconc'][:]
            asn = ds.variables['asn'][:]
            rsn = ds.variables['rsn'][:]
            sst = ds.variables['sst'][:]
            sp = ds.variables['sp'][:]
            sd = ds.variables['sd'][:]
            msl = ds.variables['msl'][:]
            blh = ds.variables['blh'][:]
            tcc = ds.variables['tcc'][:]
            u10 = ds.variables['u10'][:]
            v10 = ds.variables['v10'][:]
            t2m = ds.variables['t2m'][:]
            d2m = ds.variables['d2m'][:]
            lcc = ds.variables['lcc'][:]
            mcc = ds.variables['mcc'][:]
            hcc = ds.variables['hcc'][:]
            skt = ds.variables['skt'][:]
            swvl1 = ds.variables['swvl1'][:]
            swvl2 = ds.variables['swvl2'][:]
            swvl3 = ds.variables['swvl3'][:]
            swvl4 = ds.variables['swvl4'][:]
            stl1 = ds.variables['stl1'][:]
            stl2 = ds.variables['stl2'][:]
            stl3 = ds.variables['stl3'][:]
            stl4 = ds.variables['stl4'][:]
        sfcdict = {
                'lsm': lsm, 'siconc': siconc, 'asn': asn, 
                'rsn': rsn, 'sst': sst, 'sp': sp, 'sd': sd, 
                'msl': msl, 'blh': blh, 'tcc': tcc, 'u10': u10, 
                'v10': v10, 't2m': t2m, 'd2m': d2m, 'lcc': lcc, 
                'mcc': mcc, 'hcc': hcc, 'skt': skt, 'swvl1': swvl1, 
                'swvl2': swvl2, 'swvl3': swvl3,'swvl4': swvl4, 
                'stl1': stl1, 'stl2': stl2, 'stl3': stl3, 'stl4': stl4
                }
        return sfcdict

            
if __name__ == "__main__":
    y, m = 2018, 1
    sd = snowData()
    sd.loadingData(y, m)


In [ ]:
#if __name__ == "__main__":
mdict = [1, 2, 12]
sd = snowData()
for i in range(3):
    for m in mdict:
        yy, mm = 2018+i, m
        sd.loadingData(yy, mm)

## Data Loading from CSV-file processed

In [ ]:
data = pd.read_csv("PROCESSED_DATA", header=None)

> List of features
1.  Temperature related parameters : LR03, T2, T10, T100, T250 , T500 , T1000 , T1500 , T2000 , T2500 , T3000 , ISO_0_HGT , WLD01
2.  Humidity parameters : Q2, Q10, Q100, Q250, Q500, Q1000, Q1500, Q2000, Q2500, Q3000,
3.  Wind parameters : WS10, WS100, WS250, WS500, WS1000, WS1500, WS2000, WS2500, WS3000, 
4.  Remote sensing data parameters : CMAX


In [ ]:
#predict_data = [data[''][137, 133, ...], ....]


> Altitute according to model level ([REF](https://confluence.ecmwf.int/display/UDOC/L137+model+level+definitions))
* 2m: 
* 10m: 137 (  10.00m)
* 100m: 133 ( 106.54m)
* 250m: 129 ( 244.68m)
* 500m: 124 ( 500.01m)
* 1000m: 116 ( 987.00m)
* 1500m: 114 (1459.58m)
* 2000m: 110 (2080.41m)
* 3000m: 105 (3087.75m)

## Training for snow-detection

### DECISION TREE



In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y)

In [ ]:
tree.plot_tree(clf)

### REGRESSION



In [ ]:
reg = LinearRegression().fit(x, y)

### AUTOENCODER



In [ ]:
aut = MLPRegressor(hidden_layer_sizes = (n_encoder1, n_encoder2, n_latent, n_decoder2, n_decoder1), 
                   activation = 'tanh', 
                   solver = 'adam', 
                   learning_rate_init = 0.0001, 
                   max_iter = 20, 
                   tol = 0.0000001, 
                   verbose = True)
aut.fit(x, y)

## Evaluation

The most fundamental measures included the following: 
 * Hit Rate (HR)
 * Proportion Corrrect (PC)
 * False Alarm Ratio (FAR)
 * Critical Succcess Index (CSI) 
 * Equitable Threat Score (ETS)

\begin{align}
CSI = \frac{hits}{hits+misses+falsealarms}  
\end{align}

In [ ]:
def eval(vecY):
    for i in range(0, len(vecY)):
        if(int(vecY[i]) == 0):
            vecY[i] = 1 # label == 0, return 1
        else:
            vecY[i] = -1 # label != 0, return -1
    return vecY

\begin{align}
ETS &= \frac{hits-hits_{ranodm}}{hits+misses+false alarms-hits_{random}}  \\[1em]
hits_{random} &= \frac{(hits+misses)(guts+false alarm)}{total}  
\end{align}

## Visualization

Frequently used visualization methods included the following: 
 * 